# Code de Huffman

## Introduction: ##
Pour ce projet, nous avons choisi de nous intéresser au code de Huffman, qui permet la compression de données. 

On peut lire dans l'article de Wikipédia au 27/07/2020 : 

<i>Le codage de Huffman est un algorithme de compression de données sans perte. Le codage de Huffman utilise un code à longueur variable pour représenter un symbole de la source (par exemple un caractère dans un fichier). Le code est déterminé à partir d'une estimation des probabilités d'apparition des symboles de source, un code court étant associé aux symboles de source les plus fréquents.   

Un code de Huffman est optimal au sens de la plus courte longueur pour un codage par symbole, et une distribution de probabilité connue. Des méthodes plus complexes réalisant une modélisation probabiliste de la source permettent d'obtenir de meilleurs ratios de compression.</i>   

Il a été inventé par David Albert Huffman, et publié en 1952. 


Le codage de Huffman ne se base que sur la fréquence relative des symboles d'entrée (suites de bits) sans distinction pour leur provenance (images, vidéos, sons, etc.). Nous avons choisi dans la suite de l'appliquer à des chaînes de caractère.


In [51]:
phrase = "Le covid c'est pas fantastique, les vacances on travaille le projet"

<b>Remarque</b> : Nous avons choisi d'écrire les algorithmes en langage naturel, car le pseudo code n'est pas au programme de NSI. Dans le programme de la spécialité NSI, il est indiqué que python étant très proche du pseudo code on peut s'en passer. 

# I. Compression

### Etapes de la compression : ###
1. [obtenir les occurrences des caractères de la chaîne de caractères](#occurrence)    
2. [construire l'arbre de Huffmann](#contruction_arbre)    
   * [je propose une version en utilisant la structure "tas", c'est ce qu'on trouve classiquement ;-)](#arbre_avec_tas)
   * [une autre sans les tas, avec des listes](#arbre_avec_liste)
   
3. [calcul de la table de codage par un parcours d’arbre](#table_codage)
4. [codage](#codage)


<div id="occurrence">
<h1>1. Occurrence des caractères</h1>
    
Pour former l'arbre de Huffman, il faut déterminer l'occurrence de chacun des caractères apparaissant dans le texte.
Ci dessous, une fonction qui renvoie un dictionnaire contenant les occurrences de chacun des caractères présents d'une chaîne de caractères (les clés sont les caractères, les valeurs sont les occurrences) puis une fonction pour convertir le dictionnaire en liste, ou en tas, pour la suite

In [27]:
def creationDictionnaireoccurrence(u):
    '''Entrée : u : une chaîne de caractère e
    Sortie : dictionnaire {caractère : occurrence}'''
    occu = {}
    for c in u :
        if c in occu : #si la clé n'existe pas, on la crée et on met la valeur à 1
            occu[c]+=1
        else :
            occu[c]=1 #sinon on ajoute 1 la valeur de la clé
    return occu
    

<div class="alert alert-warning" role="alert">
Correction :   
    
  - Terminaison : boucle for qui se termine après O(n) opérations(avec n  longueur de la chaîne de caractères, car on itère sur chacun des caractères de la chaîne.
     
  - Validité  : invariant de boucle : Après k itérations, le dictionnaire contient les occurrences des caractères présents dans le début de la chaîne jusqu'au kème caractère. (Puis par récurrence...)   

In [28]:
def conversionDictionnaireoccurrenceEnListe(dico):
    '''Entrée : u : une chaîne de caractère e
    Sortie : liste contenant des tuples(occurrence,('F', caractere)'''
    liste = [(occurrence,('F', lettre)) for lettre,occurrence in dico.items()]
    return liste

<div class="alert alert-warning" role="alert">
Correction  :  
    
   - Terminaison : la conversion du dictionnaire se termine après O(n) (n longueur de la chaîne de caractère, car on itère sur chacun items du tableau, et au pire, il y a autant d'items que de caractère, si la chaîne de caractère de départ ne contient que des caractères différents.)
   
   - Validité  : invariant de boucle : Après k itérations, la liste contient les occurrences de k caractères du dictionnaire  . (Puis par récurrence...)   

In [29]:
def conversionDictionnaireoccurrenceEnTas(dico):
    '''Entrée : dico : un dictionnaire contenant des items (caractére : occurrence)
    Sortie : tas contenant des tuples(occurrence,('F', caractere)'''
    tas = [(occurrence,('F', lettre)) for lettre,occurrence in dico.items() ]#on convertit le dictionnaire en liste
    convertirListeEnTas(tas)# on "tamise" la liste pour en faire un tas (voir plus bas la notion de tas)
    return tas

<div class="alert alert-warning" role="alert">
Correction  :  
    
   - Terminaison : la conversion du dictionnaire en liste se termine après O(n) opérations (n longueur de la chaîne de caractères, car on itère sur chacun  des items du dictionnaire, et au pire, il y a autant d' items que de caractères, si la chaîne de caractères de départ ne contient que des caractères différents. Et la fonction `convertirListeEnTas` (heapify en anglais) est en O(n) aussi <a herf="#arbre_avec_tas">démonstration plus bas</a>. Donc la fonction a une complexité en O(n)
   
   - Validité : invariant de boucle : Après k itérations, la liste contient les occurrences de k caractères du dictionnaire  . (Puis par récurrence...). Voir validité de la fonction `convertirListeEnTas` <a herf="#arbre_avec_tas"> plus bas</a>
    

<div id ="construction_arbre">
    <h1> 2. La construction de l'arbre de Huffman</h1>
    

<div class="alert alert-success" role="alert">
  <h4 class="alert-heading">Comment j'ai choisi de représenter l'arbre de Huffman en Python ?</h4>


* L'arbre vide est codé par le tuple vide () .  
* les feuilles seront représentées par le t-uplet (occurrence,('F', caractère)) ( F comme "feuille").   
* Les nœuds ont un fils gauche et un fils droit . Soient G et D leurs représentations :   
les nœuds seront représentés par le t-uplet (somme des occurrences,('N', G, D)) ( N comme "nœud").

> Rq : J'ai choisi d'utiliser des t-uplets pour représenter les feuilles et les nœuds plutôt que des dictionnaires pour m'affranchir du problème lors des comparaisons.   
En effet pour "ranger le tas correctement", la fonction compare (strictement) les clés de priorité, et en cas d'égalité sur la valeur de la clé de priorité, va comparer la valeur suivante de l'élément, mais on ne peut pas comparer des dictionnaires en python ! Par contre on peut comparer des listes...

<div class="alert alert-success" role="alert">
  <h4 class="alert-heading">Algorithme de la contruction de l'arbre de Huffman</h4>
    À chaque itération, l’algorithme sélectionne les éléments  ayant les poids les plus faibles, et les assemble pour former un nouvel élément dont le poids est la somme des deux arbres. 

<div id = "arbre_avec_tas">
<h2 > Construction de l'arbre en utilisant un tas</h2>


Pour fabriquer l'arbre de Huffman associé au dictionnaire contenant les occurrences des caractères, on peut utiliser un tas.
On lit dans wikipédia :
>"En informatique, un tas (ou monceau au Canada, heap en anglais) est une structure de données de type arbre qui permet de retrouver directement l'élément que l'on veut traiter en priorité.  

C'est ce que l'on veut dans la construction de l'arbre de Huffman !

En effet un arbre de Huffmann est un arbre binaire, et on veut pouvoir traiter les nœuds de poids (occurrence) faibles. Ce que permet typiquement un tas.

>On dit qu'un arbre est ordonné en tas lorsque la propriété suivante est vérifiée :
>
>    Pour tous nœuds A et B de l'arbre tels que B soit un fils de A
>    clé(A) ≥ clé(B)
>
>ou
>
>    Pour tous nœuds A et B de l'arbre tels que B soit un fils de A
>    clé(A) ≤ clé(B)
>
>Un arbre vérifiant cette propriété est aussi appelé « arbre tournoi ». Cette propriété implique que la plus grande clé (ou la plus petite) soit située à la racine du tas. Ils sont ainsi très utilisés pour implémenter les files à priorités car ils permettent des insertions en temps logarithmique et un accès direct au plus grand élément. L'efficacité des opérations effectuée sur des tas est très importante dans de nombreux algorithmes sur les graphes. Les tas sont en outre utilisés dans l'algorithme de tri par tas.
>
>On peut ainsi représenter un tas à n éléments par un tableau à n cases, de la façon suivante :
>
>Si on note 0 le numéro de la racine du tas, on peut définir récursivement le numéro de tous les sommets dans l'arbre, grâce à la formule : Pour un sommet parent numéroté i, son fils gauche aura pour numéro 2*i+1, et son sommet droit 2*i+2. Le parent d'un sommet j a donc toujours pour numéro la partie entière inférieure de (j-1)/2.   

<img src="heap.png">

<a href ="https://towardsdatascience.com/data-structure-heap-23d4c78a6962">source</a>  

Un tas est une structure de données permettant les opérations suivantes :  
- Mettre dans le tas un objet avec une priorité donnée (une priorité est, par exemple, un entier, ici ce sera l'occurrence ou la somme des occurrences des fils)
- Retirer de la file l'objet de priorité minimale.


> <b>Remarque</b> : Pour le tri dans le tas    
A valeur de clé de priorité (occurrence (ou somme des occurrences)) égale, l'emploi de p-uplet permet bien l'ordonnancement car, ce sera F avant N (par ordre alphabétique), et pour deux feuilles, ordre alphabétique du caractère, pour deux nœuds ordre alphabétique du fils gauche... ). Le caractère étant unique dans l'arbre, l'ordonnancement sera assuré.  

<div class="alert alert-danger" role="alert">
Voici l'algorithme, pour la construction de l'arbre en utilisant les tas :   
     
1. On met dans le tas les futures feuilles de notre arbre avec leurs occurrences comme clé de priorité. Pour cela il suffit de lire le dictionnaire des occurrences dont les clés sont les lettres et les valeurs sont les occurrences. (Fait en haut dans la fonction conversionDictionnaireOccurrenceEnTas)   
2. Pour k allant de 0 au nombre de feuilles -1 :  (il y a n feuilles, il y aura n-1 nœuds binaires) 
   * On retire du tas la racine ( et on réarrange le tas), soit x cet élément. 
   * On retire du nouveau tas la racine ( et on réarrange le tas), soit y cet élément. 
   * On crée le nœud z dont les deux fils sont x et y, et de priorité la somme des priorités des fils .   
   * On met z dans le tas correctement rangé.   
3. Maintenant, le tas contient un unique arbre. On renvoie cet arbre.   

Il faut donc :
 * une fonction pour transformer une liste en tas, c'est ce que fait la fonction `convertirListeEnTas` 
 * une fonction pour retirer la racine du tas (occurrence la plus petite) et réarranger le tas, c'est ce que fait la fonction `suppressionDeLaRacine`
 * une fonction pour ajouter un nœud dans le tas et réarranger le tas, c'est ce que fait la fonction `insererElement`
 

### Les fonctions nécessaires pour manipuler les tas binaires ###


Fonction `suppressionDeLaRacine` (c'est à dire du min)        
  + entrée : une liste qui implémente un tas de n éléments
  + sortie : une liste qui implémente un tas constitué des n-1 éléments du tas d'entrée moins sa racine **et** la racine supprimée

**Algorithme**
1. On remplace la racine (c'est à dire l'élément 0 de la liste) par le dernier élément du tas 
2. on fait descendre cet élément dans le tas, en réagençant les différents éléments, pour garder la structure de tas.

Pour l'étape 2, il nous faut une procédure `descendre` (de la racine vers les feuilles)
  + Entrée : une liste qui implémente un tas, sauf 1 élément d'indice i **et** l'indice i
  + Sortie : une liste qui implémente un tas
  
**Algorithme**   
Tant que l'on a pas atteint l'avant dernier étage (indice dans la liste <n//2) ou que l'élément n'est pas plus petit que ces deux enfants   
  * On compare l'élément i à ses deux enfants (2*i et 2*i+1)
  * Si l'élément i est plus grand qu'un de ses enfants (ou les deux) on l'échange avec le plus petit de ses enfants et i prend la valeur de l'indice de l'enfant avec qui on l'a échangé


In [30]:
def descendre (presqueTas,i): #on descend l'élément i vers les feuilles jusqu'à ce que l'élément soit dans une position de parent qui respecte la structure de tas
    '''Entrée : une liste qui implémente un tas, sauf 1 élément d'indice i 
  Sortie : une liste qui implémente un tas'''
    n = len(presqueTas)
    
    while i<n//2 : #on descends juste avant les feuilles
        indiceEnf1 = 2*i+1 #les enfants sont en 2*i+1 et 2*i+2 le deuxieme enfant n 'existe pas forcement'
        enfant1 = presqueTas[indiceEnf1]
        if indiceEnf1<(n-1) : #cela signifie qu 'il y a le deuxieme enfant'
            indiceEnf2 = 2*i+2 
            
            enfant2 = presqueTas[indiceEnf2]
            if presqueTas[i]>enfant1 or presqueTas[i]>enfant2:
                if (enfant2>enfant1): #on échange avec le plus petit des enfants
                    presqueTas[i],presqueTas[indiceEnf1] = presqueTas[indiceEnf1],presqueTas[i]
                    i =indiceEnf1
                    
                else :
                    presqueTas[i],presqueTas[indiceEnf2] = presqueTas[indiceEnf2],presqueTas[i]
                    i =indiceEnf2
                    
                
            else : break  
        else : #il n'y a qu'un enfant on vérifie si il faut faire l'échange ou pas 
            if presqueTas[i]>enfant1:
                    presqueTas[i],presqueTas[indiceEnf1] = presqueTas[indiceEnf1],presqueTas[i]
                    i =indiceEnf1
                    
            else: break
            
   
    
def suppressionDeLaRacine(tas):
    '''entrée : une liste qui implémente un tas de n éléments
   sortie : une liste qui implémente un tas constitué des n-1 éléments du tas d'entrée moins sa racine et la racine supprimée'''
    racine = tas[0]
    tas[0]=tas[len(tas)-1]
    tas.pop()
    descendre (tas,0)
    return (racine)



#### Pour tester : 
if True :
    tas = [3,4,6,21,10,7,8,22,28,13,19,25,20]
    print (tas)
    for i in range(len(tas)):
        suppressionDeLaRacine(tas)
        print (tas)

[3, 4, 6, 21, 10, 7, 8, 22, 28, 13, 19, 25, 20]
[4, 10, 6, 21, 13, 7, 8, 22, 28, 20, 19, 25]
[6, 10, 7, 21, 13, 25, 8, 22, 28, 20, 19]
[7, 10, 8, 21, 13, 25, 19, 22, 28, 20]
[8, 10, 19, 21, 13, 25, 20, 22, 28]
[10, 13, 19, 21, 28, 25, 20, 22]
[13, 21, 19, 22, 28, 25, 20]
[19, 21, 20, 22, 28, 25]
[20, 21, 25, 22, 28]
[21, 22, 25, 28]
[22, 28, 25]
[25, 28]
[28]
[]


<div class="alert alert-warning" role="alert">
Correction de la fonction descendre:  
    
   - Terminaison : 
     - Descendre se terminera après O(log n) opérations car au pire on fera descendre de la racine jusqu'à une feuille l'élément, c'est à dire de toute la hauteur de l'arbre -1 = h-1 qui est O(log n). (voir démo plus bas) 
     - Une autre façon de voir, est de définir le convergent : n//2 - i,i, est au pire des cas initialisé à 0, et i est multiplié par 2 à chaque itération, donc la boucle se termine, et au pire il y aura log(n) itérations.
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : A la fin d'une itération, les éléments de la liste autre que i, 2.i+1 et 2.i+2 ont bien une structure de tas, c'est à dire que le parent est inférieur à ces deux enfants.
     - Par récursivité :
       - La première itération, la liste est rangée en tas (par précondition de l'entrée), sauf peut être i et ses enfants, lors de la première itération, on "range" en structure de tas i et ses enfants, et donne à i la valeur de l'indice de l'enfant avec qui il a été échangé. 
       - A chaque tour de boucle, on range correctement, i et ses enfants pour respecter la structure de tas, puis i prend la valeur de l'indice de l'enfant avec qui on l'a échangé, donc, la structure d'arbre est respectée, sauf pour le nouveau i et ses enfants.
       - On sort de la boucle, soit si i est bien rangé, et comme tous les autres éléments sont bien rangés, la liste a bien une structure de tas. Soit si i>=n//2, c'est à dire si l'élément auquel on s'intéresse est une feuille. (i>=n//2 ---> 2*i+1 >= n+1 donc il n'y a pas de fils pour l'élément i, et l'élément i est une feuille)
       
> Rq : la correction de la fonction suppressionDeLaRacine, dépend directement de la correction de la fonction descendre

<b>Remarque</b> : hauteur d'un tas : h (un tas est un arbre binaire presque complet)   
Un tas T qui emmagasine n éléments a une hauteur h = O(log n)   
Preuve:      
– Soit h la hauteur d'un tas stockant les n  clés   
– Puisque il y a 2 ** i clés à la profondeur i= 0, ... , h - 2 et au moins une clé à profondeur h - 1, nous avons $ n ≥ 1 + 2 + 4 +...+ 2^{h-2}  + 1  $ 
–Ainsi, $ n ≥  2^{h-1} $  , i.e., h ≤ log(n) + 1


Fonction `insererElement`         
  + entrée : une liste qui implémente un tas de n éléments **et** un élément à ajouter.
  + sortie : une liste qui implémente un tas constitué des n éléments du tas d'entrée et de l'élément à ajouter

**Algorithme**
1. On place l'élément à la fin de la liste 
2. on fait monter cet élément dans le tas (vers la racine), en réagençant les différents éléments, pour garder la structure de tas.

Pour l'étape 2, il nous faut une procédure `monter` (des feuilles vers la racine)
  + Entrée : une liste qui implémente un tas, sauf le dernier élément
  + Sortie : une liste qui implémente un tas
  
**Algorithme**   
Tant que l'on a pas atteint la racine (i>=1)  ou que l'élément n'est pas plus grand que son parent   
  * On compare l'élément i à son parent d'indice ((i-1)//2)
  * Si l'élément i est plus petit que son parent on l'échange avec le plus petit de ses enfants, et i prend  la valeur de l'indice du parent

In [31]:
def rajouterElement (tas, e):
    '''entrée : une liste qui implémente un tas de n éléments et un élément à ajouter.
  sortie : une liste qui implémente un tas constitué des n éléments du tas d'entrée et de l'élément à ajouter'''
    tas.append(e)
    monter(tas)
    return tas

def monter (tas): #on monte le dernier élément jusqu'à une bonne place, vers la racine
    '''Entrée : une liste qui implémente un tas, sauf le dernier élément
  Sortie : une liste qui implémente un tas'''
    i = len(tas)-1
    while i>=1:
        indiceParent = (i-1)//2
        if tas[i]<tas[indiceParent] :
            tas[i],tas[indiceParent] = tas[indiceParent],tas[i]
            i = indiceParent
        else :
            break

#### Pour tester :
if True :            
    tas = [3,4,6,21,10,7,8,22,28,13,19,25,20]
    rajouterElement(tas,2)
    print (tas)   

[2, 4, 3, 21, 10, 7, 6, 22, 28, 13, 19, 25, 20, 8]


<div class="alert alert-warning" role="alert">
Correction de la fonction monter:  
    
   - Terminaison : 
     - Monter se terminera après O(log n) opérations car au pire on fera monter des feuilles jusqu'à la racine l'élément, c'est à dire de toute la hauteur de l'arbre -1 = h-1 qui est O(log n).  
     - Une autre façon de voir, est de définir le convergent : i-3, i est  initialisé à la n (nombre d'élément), et i est divisé par 2 à chaque itération, donc la boucle se termine, et au pire il y aura log(n) itérations.
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : A la fin d'une itération, les éléments de la liste autre que i et son parent d'indice (i-1)//2 ont bien une structure de tas, c'est à dire que le parent est inférieur à ces deux enfants.
     > Rq : Et le deuxième fils !
>    - soit le dernier élément enf1 a un frère enf2, et son frère est supérieur à son parent par, car la structure de tas est respectée donc enf2>par, si on effectue l'échange entre enf1 et parent, c'est que enf1<par, ce qui implique enf2>enf1 (or enf1 devient le parent de enf2) la structure de tas est respectée. 
>    - Si le dernier élément est fils unique, la question du deuxième enfant ne se pose pas !
> Cela ne se pose que pour le dernier élément, car un tas est un arbre binaire presque complet.
     - Par récursivité :
       - La première itération, "range", si nécessaire, en structure de tas le dernier élément et son parent (pour le deuxième enfant voir remarque ci-dessus), et donne à i la valeur de l'indice du parent avec qui il a été échangé. 
       - A chaque itération, on range correctement, i et son parent pour respecter la structure de tas, puis i prend la valeur de l'indice du parent avec qui on l'a échangé, donc, la structure d'arbre est respectée, sauf pour le nouveau i et son parent.
       - On sort de la boucle, soit si i est bien rangé, et comme tous les autres éléments sont bien rangés, la liste a bien une structure de tas. Soit si i=0, c'est à dire si l'élément auquel on s'intéresse est une racine. 
       
> Rq : la correction de la fonction rajouterElement, dépend directement de la correction de la fonction monter

Fonction `convertirListeEnTas`         
  + entrée : une liste à n éléments (avec une clé de priorité)
  + sortie : une liste qui implémente un tas constitué des n éléments de la liste
  
Nous pourrions choisir d'insérer les n éléments un par un (insérerElément en O(log n ), pour chacun de n éléments), mais cela ferait O(n log(n)) opérations.

Il y a mieux !  en O(n)

L'idée est de réarranger récursivement chaque sous arbre dans le tas en commençant par les feuilles


**Algorithme**
On parcourt les éléments en partant de l'avant dernier niveau (donc le niveau qui ne contient pas de feuille, soit d'indice = à n//2 et on remonte j'usqu' à la racine de droite à gauche (donc n//2, n//2-1, n//2-2....0).
    chaque élément est descendu vers les feuilles, jusqu'à ce qu'il soit à une position qui respecte la structure de tas.


In [32]:
def convertirListeEnTas(x):
    """
    entrée : une liste à n éléments (avec une clé de priorité)
    sortie : une liste qui implémente un tas constitué des n éléments de la liste
"""
    n = len(x)
    # Transform bottom-up.  The largest index there's any point to looking at
    # is the largest with a child index in-range, so must have 2*i + 1 < n,
    # or i < (n-1)/2.  If n is even = 2*j, this is (2*j-1)/2 = j-1/2 so
    # j-1 is the largest, which is n//2 - 1.  If n is odd = 2*j+1, this is
    # (2*j+1-1)/2 = j so j-1 is the largest, and that's again n//2-1.
    for i in reversed(range(n//2)):
        descendre(x, i)
#### Pour tester :
if True :
    tas = [1,2,45,8,9,7,6]
    convertirListeEnTas(tas)
    print (tas)

[1, 2, 6, 8, 9, 7, 45]


<div class="alert alert-warning" role="alert">
Correction de la fonction convertirListeEnTas:  
    
   - Terminaison : 
     - Boucle for, qui s'effectue n//2 fois, (et la fonction descendre se termine)
     - Complexité : Soit h la hauteur de l'arbre, on numérote les niveau de 0 pour la racine à h-1 pour les feuilles.   
       pour un élément d'un niveau i, il pourra au max descendre de h-i-1 niveaux jusqu'à une feuille. Et pour un niveau i, il y $ 2^i $ éléments. Donc au max, il y aura :  
       
     $ C = \sum_{i=0}^{h-1} (h-i-1). 2^i $ opérations   
     
     On effectue le changement de variable $ j = h-i-1 $
     
     $ C = 2^{h-1} \sum_{j=1}^{h-1} \frac 1 {2^j} $ opérations  
     
     Or $ \sum_{j=1}^{h-1} \frac 1 {2^j} $ converge (série géométrie de raison inf à 1)  
     
     Donc $ C = O (2^{h-1}) $   
     
     Or h <= log n  
     
     Donc C = O(n)
     
     
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : Pendant le parcours des éléments on maintient l'invariant suivant : Tous les sous-arbres des élément à droite de l'élément courant (sur le même niveau) sont des tas. A chaque itération, on applique à l'élément courant, la fonction descendre qui place l'élément tel que le sous arbre sous l'élément courant soit un tas. Donc après avoir traité la racine, comme elle vérifie l'invariant, notre arbre est un tas.
       
> Rq : la correction de cette fonction, dépend directement de la correction de la fonction descendre faite plus haut

In [33]:

def faireArbre(tas, aff=False):#aff pour l'affichage
    n = len(tas)
    if aff : print('Tas initial :', tas)
    for k in range(n - 1): # il y a n feuilles, il y aura n-1 nœuds binaires
        x = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurrence) et on réordonne le tas 
        y = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurrence) et on réordonne le tas 
        ## les deux t-uplets précédents contiennent les 2 arbres de plus petit point du tas 
        z = ('N', x, y) #on crée le nœud
        rajouterElement(tas, (x[0] + y[0], z))#on l'ajoute au tas, et on réarrange le tas
        if aff: print('Étape %d : %s' % (k, tas))
    return suppressionDeLaRacine(tas) #on extrait l'arbre de huffman. A la fin le tas est composé d'une liste avec pour premiere valeur la somme de toutes les occurrences, et en deuxième (indice 1) l'arbre proprement dit
#### Pour tester :
if True :
    phrase2 ="abracadabra" #pour plus de facilité, une phrase courte, possédant peu de caractére différents
    dicOccu=creationDictionnaireoccurrence(phrase2)
    tas = []
    tas = conversionDictionnaireoccurrenceEnTas(dicOccu)
    print(tas)
    
    A = faireArbre(tas,True)
    print (A)

[(1, ('F', 'c')), (1, ('F', 'd')), (2, ('F', 'r')), (2, ('F', 'b')), (5, ('F', 'a'))]
Tas initial : [(1, ('F', 'c')), (1, ('F', 'd')), (2, ('F', 'r')), (2, ('F', 'b')), (5, ('F', 'a'))]
Étape 0 : [(2, ('F', 'b')), (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (2, ('F', 'r')), (5, ('F', 'a'))]
Étape 1 : [(2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (5, ('F', 'a')), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))]
Étape 2 : [(5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))]
Étape 3 : [(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))]
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))


<div class="alert alert-warning" role="alert">
Correction de la fonction faireArbre:  
    
   - Terminaison : 
     - Boucle for, qui s'effectue n fois, (et les fonctions suppressionDeLaRacine et rajouterElement se terminent)
     - Complexité : La boucle s'exécute n fois, et  les fonctions suppressionDeLaRacine et rajouterElement sont en O(log n), donc faireArbre est en O(n log(n))
     
     

   - Validité : 
      - L'algorithme est bien celui de Huffman, pour ce qui est de la démonstration que le code de huffman est optimal, on trouve la démonstration dans la littérature, et je ne la recopierai pas ici.
       


### Construction de l'arbre en utilisant des listes et sans les tas ###

Cet algorithme ressemble directement à la méthode employée lorsqu'on fait l'arbre d'Huffman " à la main".  

#### Les fonctions nécessaires pour manipuler les tas binaires ####
fonction recherche du min   
fonction supprimer des éléments d'une liste (on utilisera pop)     
fonction associer deux nœuds     
fonction ajouter un élément dans la liste (append)    

#### Algorithme :
tant que la liste contient plus de 1 élément :
   on recherche les deux plus petits éléments de la liste
   on les supprime de la liste
   on les associe, on les fusionne
   on les place à la fin de la liste
   
par récursivité :
  si le nombre d'élément de la liste est = à 1, on retourne la liste
  sinon on appelle la fonction avec la liste dans laquelle on a supprimé les deux plus petits éléments et rajouté, la fusion des deux éléments supprimés

In [34]:
def mini (liste):
    '''entrée : liste contenant des tuples(occurrence,('F', caractere)) ou (occurrence,('N', e1, e2))si feuille ou noeud
    sortie : l élement de poids (occurence) la plus petite, et son indice '''
    mini = liste[0][0]
    indiceDuMin =0
    for i in range (1,len(liste)):
        if mini>liste[i][0]:
            mini = liste[i][0]
            indiceDuMin = i
            print (indiceDuMin)
    return liste[indiceDuMin],indiceDuMin

def supprimerElemeParIndice (liste,i) :
    '''supprime l élément i de la liste'''
    liste.pop(i)
    
def fusion (e1,e2):
    '''entrée : deux éléments du type (occurrence,('F', caractere)) ou (occurrence,('N', elem1, elem2))
    sortie : un nouvel élément du type (somme_occurrence,('N', e1, e2))'''
    e3occu = e1[0]+e2[0]
    return(e3occu ,('N', e1,e2))

def ajouterElement (liste,e) :
    '''ajoute l élément e à la fin de la liste'''
    liste.append(e)

In [35]:
from copy import deepcopy
def faireArbreAvecListe (listeOccu):
    liste = deepcopy(listeOccu)
    while (len(liste)>1) :
        elem1,indiceElem1 = mini (liste)
        print('elem1 ',elem1,indiceElem1)
        supprimerElemeParIndice (liste,indiceElem1)

        print (listeOccu)

        elem2,indiceElem2 = mini (liste)
        print('elem2 ',elem2,indiceElem2)
        supprimerElemeParIndice (liste,indiceElem2)
        print (liste)

        elem3 = fusion(elem1,elem2)


        print('elem3 ' ,elem3)
        ajouterElement (liste,elem3)
    return liste[0]

phrase2 ='abracadabra'
listeOccu=conversionDictionnaireoccurrenceEnListe(creationDictionnaireoccurrence(phrase2))
arbre = faireArbreAvecListe(listeOccu)
print (arbre)
print (listeOccu)

1
3
elem1  (1, ('F', 'c')) 3
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
3
elem2  (1, ('F', 'd')) 3
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r'))]
elem3  (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))))
1
elem1  (2, ('F', 'b')) 1
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
elem2  (2, ('F', 'r')) 1
[(5, ('F', 'a')), (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))))]
elem3  (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))
1
elem1  (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))) 1
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
elem2  (4, ('N', (2, ('F', 'b')), (2, ('F', 'r')))) 1
[(5, ('F', 'a'))]
elem3  (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))
elem1  (5, ('F', 'a')) 0
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
elem2  (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))

<div class="alert alert-warning" role="alert">
Correction de la fonction faireArbreAvecListe:  
    
   - Terminaison : 
     - La taille de la liste diminue à chaque itération, donc la boucle se termine
   - Complexité : 
     1. La copie de la liste est en O(n)  
     2. A chaque itération : 
         - extraire 2 fois l’arbre de poids min, (il faut parcourir toute la liste-qui certe diminue à chaque itération)
         - fusionner pour créer un nouvel arbre à partir des 2 précédents :O(1)
         - rajouter ce nouvel arbre à la fin. 
     3. n itérations  
     La complexité est donc en O(n^2)
     
     

   - Validité : Il faut vérifier qu'à la fin, la liste contient bien 1 seul élément constitué de l'arbre de Huffman
     - Invariant de boucle : A la fin d'une itération, le dernier éléménet de la liste est contitué de l'arbre d'Huffman correspondant aux k caractéres les moins présents dans le texte.
    
     - Par récursivité :
       - La première itération, fusionne les deux caractéres les moins présents et range ce nouvel élément à la fin de la liste   
       - A chaque itération, on fusionne les deux éléments de poids les plus faibles, et ajoute le nouvel élément à la fin.La longueur de la liste diminue de 1
       - On sort de la boucle, lorsqu'elle ne contient plus qu'un élément, qui contient bien l'arbre de Huffman.
        
      
**La construction de l’arbre de Huffman avec des listes et sans utiliser les tas est donc en O(n^2) alors qu'avec les tas, elle est en O(n.log(n))**


<div id ="table_codage">
    <h1> 3. Construction du dictionnaire (table de codage)</h1>
  

Pour construire le dictionnaire qui à chaque caractère associe le code binaire, il faut parcourir l'arbre. Le parcours se fait en profondeur, et est dans l'ordre préfixe. On s'arréte quand on trouve une feuille, le chemin suivi correspond au code du caractére.

In [36]:


def parcours(arbre,chemin,code) :
    '''Entrée : arbre binaire (sous arbre de l'arbre initial), chemin (binaire) suivi dans l'arbre initial jusqu'au "sous arbre",
    code, contient le dictionnaire qui se remplit lorsqu une feuille est atteinte'''
    
    # sur une feuille, on peut associer ce qui est dans chemin à la lettre (et on remplit le dictionnaire)
    if arbre[1][0] =='F' :
        
        code[arbre[1][1]] = chemin
        return
    
    # sinon, on parcourt l'arbre en mémorisant le chemin pris dans la variable chemin
    
    parcours(arbre[1][1],chemin+'0',code) #fils de gauche
    
    parcours(arbre[1][2],chemin+'1',code)#fils de droite

def code_huffman(arbre) :
    # on remplit le dictionnaire du code d'Huffman en parcourant l'arbre
    code = {}
    if len(arbre[1])>2 :
        parcours(arbre,'',code)
        
    else : #Si le texte ne comporte qu'un seul caractère
        code[arbre[1][1]] = '0'
    return code
phrase ='a'
listeOccu=conversionDictionnaireoccurrenceEnTas(creationDictionnaireoccurrence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')

(1, ('F', 'a'))
Dico =
 {'a': '0'} 



<div class="alert alert-warning" role="alert">
Correction de la fonction :  
    
   - Terminaison : 
     - L'algorithme est ici récursif, le parcours se fait en préfixe, jusqu'à chacune des feuilles, et se termine
     - Complexité : 
        - Pour aller de la racine à la feuille, il y a au max, un nombre de noeud correspondant à la hauteur de l'arbre. Or la hauteur de l'arbre est en O(log(n)).
        - Le nombre de caractère est en O(n).
        - Donc la créaction du dictionnaire est en O(n log(n))
     
     

   - Validité : le dictionnaire est rempli au fur et à mesure, que l'on trouve une feuille. Toutes les feuilles sont atteintes.
    


<div id ="codage">
    <h1> 4. Encodage d'un texte à partir du dictionnaire</h1>

A partir d'un fichier 

In [37]:
def encodageFichier(dico,fichier) :
    
    f_in = open(fichier,'r',encoding='utf-8')
    texte = f_in.read()
    f_in.close()
    
    caracteresPresentsDico = [c for c in dico.keys()]
    binaire = ''
    for car in texte :
        if car in caracteresPresentsDico :
            binaire = binaire + dico[car]
        else :
            binaire = binaire + dico[' ']
    return binaire

A  partir d'un texte contenu dans une variable :

In [38]:
def encodage(dico,texte) :

    caracteresPresentsDico = [c for c in dico.keys()]
    binaire = ''    
    for car in texte :
        if car in caracteresPresentsDico :
            binaire = binaire + dico[car]
        else :
            binaire = binaire + dico[' ']
    return binaire
phrase ='abracadabra'
listeOccu=conversionDictionnaireoccurrenceEnTas(creationDictionnaireoccurrence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')
binaire = encodage(dico,phrase)
print(binaire)

(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))
Dico =
 {'a': '0', 'c': '100', 'd': '101', 'b': '110', 'r': '111'} 

01101110100010101101110


<div class="alert alert-warning" role="alert">
Complexité de la fonction :  
    Le texte est parcouru caractère, par caractère, donc la complexité est en O(n).   
    L'opérateur in pour un dictionnaire est en moyenne en O(1) (voir <a href ="https://wiki.python.org/moin/TimeComplexity">ici</a>)

### Conclusion sur la complexité.###
<div class="alert alert-warning" role="alert">


**Les différentes étapes en utilisant la structure de tas, sont toutes en O(n) ou en O(n log(n)), donc la totalité
des opération est en O(n log(n)).**
    

# II. Décompression

## 1. Décodage d'un fichier compressé

Il faut fournir l'arbre de Huffman, car il n'est pas unique (de manière générale, ici l'implémentation faite fourni le même arbre à chaque fois, mais de manière générale il existe plusieurs arbres de Huffman pour une même entrée).

In [39]:
def decodageFichier(arbre,fichierCompresse) :

    # recopie du contenu du fichier
    f_in = open(fichierCompresse,'r',encoding='utf-8')
    compresse = f_in.read()
    f_in.close()

    huffman = arbre
    texte = ''

    # parcours de l'arbre d'Huffman en fonction du bit lu 
    for bit in compresse :
        
        if bit == '0' :
            arbre = arbre[1] #fils de gauche
        else : 
            arbre = arbre[2] #fils de droite

        # sur une feuille, on décode une lettre et on repart à la racine
        if arbre[0] =='F' :
            texte = texte + arbre[1]
            arbre = huffman
    return texte

In [40]:
def decodage(arbre,binaire) :

    huffman = arbre
    texte = ''

    # parcours de l'arbre d'Huffman en fonction du bit lu 
    for bit in binaire :
        print (bit)
        if bit == '0' :
            arbre = arbre[1][1] #fils de gauche
        else : 
            arbre = arbre[1][2] #fils de droite

        # sur une feuille, on décode une lettre et on repart à la racine
        if arbre[1][0] =='F' :
            texte = texte + arbre[1][1]
            arbre = huffman
    return texte
phrase ="Codage d'un texte un petit peu plus long et plus complexe à coder."
listeOccu=conversionDictionnaireoccurrenceEnTas(creationDictionnaireoccurrence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')
binaire = encodage(dico,phrase)
print(binaire)
print(decodage (arbre,binaire))

(66, ('N', (28, ('N', (12, ('F', ' ')), (16, ('N', (8, ('N', (4, ('F', 'l')), (4, ('F', 'o')))), (8, ('N', (4, ('N', (2, ('F', 'c')), (2, ('F', 'g')))), (4, ('N', (2, ('F', 's')), (2, ('F', 'x')))))))))), (38, ('N', (17, ('N', (8, ('N', (4, ('N', (2, ('N', (1, ('F', "'")), (1, ('F', '.')))), (2, ('N', (1, ('F', 'C')), (1, ('F', 'a')))))), (4, ('N', (2, ('N', (1, ('F', 'i')), (1, ('F', 'm')))), (2, ('N', (1, ('F', 'r')), (1, ('F', 'à')))))))), (9, ('F', 'e')))), (21, ('N', (10, ('N', (5, ('F', 'p')), (5, ('F', 't')))), (11, ('N', (5, ('F', 'u')), (6, ('N', (3, ('F', 'd')), (3, ('F', 'n'))))))))))))
Dico =
 {' ': '00', 'l': '0100', 'o': '0101', 'c': '01100', 'g': '01101', 's': '01110', 'x': '01111', "'": '100000', '.': '100001', 'C': '100010', 'a': '100011', 'i': '100100', 'm': '100101', 'r': '100110', 'à': '100111', 'e': '101', 'p': '1100', 't': '1101', 'u': '1110', 'd': '11110', 'n': '11111'} 

10001001011111010001101101101001111010000011101111100110110101111110110100111011111001100101

<div class="alert alert-warning" role="alert">
<b>Correction :</b> 
    
<b> -Terminaison:</b>  boucle for qui se termine après n itérations (avec n nombres de bits du message codé).

<b> -Validité:</b>  Le parcours de l’arbre se termine toujours par une feuille et dans notre cas, le caractère est toujours présent dans l’arbre.

<b> Complexité :</b> 

Il y a 2 comparaisons et 3 affectations par passage dans la boucle for. On a donc une complexité O(n) = 5n complexité linéaire.
Pour mieux comprendre ce qui se passe, on peut par exemple faire apparaître au fur et à mesure le décodage.

## 2. Exemple illustré
Prenons une phrase assez courte : 
<center><b>« en juillet, encore le projet »</b></center>

Le codage donne comme sortie le code suivant : 
<center><b>11010100001110011101111100010010001010111010110010001111010111100000110100001111111110010111001001010</b></center>

Avec l’arbre généré par la fonction qui sera :
('N',  (12,   ('N',    (6, ('F', 'e')),    (6,     ('N', (3, ('F', 'l')), (3, ('N', (1, ('F', 'p')), (2, ('F', 'n')))))))),  (16,   ('N',    (8, ('N', (4, ('F', ' ')), (4, ('N', (2, ('F', 'j')), (2, ('F', 't')))))),    (8,     ('N',      (4, ('N', (2, ('F', 'o')), (2, ('F', 'r')))),      (4,       ('N',        (2, ('N', (1, ('F', 'u')), (1, ('F', 'i')))),        (2, ('N', (1, ('F', ',')), (1, ('F', 'c'))))))))))))

Qu’il est plus simple pour nous de mettre sous forme d’un tableau (il s'agit du dicoSortie):

|lettre|e | l | p  |  n |‘ ‘|  j | t  | o  | r  |   u  |  i  |  ,  |  c  |
|------|--|---|----|----|---|----|----|----|----|------|---- |-----|-----|
| code |00|010|0110|0111|100|1010|1011|1100|1101|111000|11101|11110|11111|


En modifiant la fonction initiale, il est possible de faire apparaître le travail de décodage 

In [41]:
def decodageExplicite(arbre,binaire) :
    huffman = arbre    
    texte=''
    code=list(binaire) #converti en liste pour supprimer des éléments
    for bit in binaire : 
        nbBit=0
        if bit == '0' :
            arbre = arbre[1][1] #fils de gauche
        else : 
            arbre = arbre[1][2] #fils de droite
    
        # sur une feuille, on décode une lettre et on repart à la racine
        if arbre[1][0] =='F' :
            texte = texte + arbre[1][1]
            nbBit=len(dicoSortie[arbre[1][1]])        
            arbre = huffman        
            for i in range(0,nbBit):
                del code[0]
            sortie= "".join(code)
            print(texte,"--",sortie)
    return texte

"""
texte="en juillet, encore le projet"
dico=creationDictionnaireOccurence(texte)
liste=conversionDictionnaireOccurenceEnListe(dico)
arbre = faireArbreAvecListe(liste)
dicoSortie = code_huffman(arbre)
binaire = encodage(dicoSortie,texte)
"""
arbre=(28, ('N',(12,('N',(6, ('F', 'e')),(6,('N', (3, ('F', 'l')), (3, ('N', (1, ('F', 'p')), (2, ('F', 'n')))))))),(16,('N',(8,('N',(4,('F',' ')),(4,('N', (2,('F', 'j')), (2, ('F','t')))))),
    (8,('N',(4, ('N', (2, ('F', 'o')), (2, ('F', 'r')))),(4,('N',(2, ('N', (1, ('F', 'u')), (1, ('F', 'i')))),(2, ('N', (1, ('F', ',')), (1, ('F', 'c'))))))))))))
dicoSortie={'e': '00', 'l': '010', 'p': '0110', 'n': '0111', ' ': '100', 'j': '1010', 't': '1011', 'o': '1100', 'r': '1101', 'u': '11100', 'i': '11101', ',': '11110', 'c': '11111'}
binaire='0001111001010111001110101001000101111110100000111111111100110100100010001000110110111001010001011'
decodageExplicite(arbre,binaire)

e -- 01111001010111001110101001000101111110100000111111111100110100100010001000110110111001010001011
en -- 1001010111001110101001000101111110100000111111111100110100100010001000110110111001010001011
en  -- 1010111001110101001000101111110100000111111111100110100100010001000110110111001010001011
en j -- 111001110101001000101111110100000111111111100110100100010001000110110111001010001011
en ju -- 1110101001000101111110100000111111111100110100100010001000110110111001010001011
en jui -- 01001000101111110100000111111111100110100100010001000110110111001010001011
en juil -- 01000101111110100000111111111100110100100010001000110110111001010001011
en juill -- 00101111110100000111111111100110100100010001000110110111001010001011
en juille -- 101111110100000111111111100110100100010001000110110111001010001011
en juillet -- 11110100000111111111100110100100010001000110110111001010001011
en juillet, -- 100000111111111100110100100010001000110110111001010001011
en juillet,  -- 0001111111111001101001000100

'en juillet, encore le projet'

# III Analyse des résultats obtenus


### 1. Rapport entre le texte initial et le texte compressé
Pour voir à l’usage la qualité du programme fait, j’ai comparé le taux de compression réalisé par le programme pour différents fichiers texte.
Pour commencer j’ai pris des romans en français de différentes longueurs.
La compression est définie comme le rapport entre le nombre de bits que fait le texte compressé et celui de texte initial.

|Roman|Taille initiale(bits)|Taille compressée(bits)|Nombre de caractères différents|Codage sur nombre de bits|Taux de compression|
|:---:|:-------------------:|:---------------------:|:-----------------------------:|:-----------------------:|:-------------------:|
|4 fables|36360|20416|72|2 à 12|56 %|
|Candide|1611640|917975|95|3 à 17|56 %|
|Le Horla|2130432|1231817|101|3 à 18|57 %|
|Intégrale des fables|4272944|2378855|108|2 à 20|55 %|
|Guerre et Paix|23187896|13177059|110|3 à 21|56 %|
|Les misérables|26010384|14838532|119|3 à 22|57 %|
|3 tomes de la comédie humaine|27394856|15409650|109|3 à 21|56 %|

La taille du texte ne semble pas avoir d’importance sur le taux de compression.
Pour le vérifier expérimentalement j’ai créé une fonction qui prend un texte le coupe en différentes tailles, et je vérifie graphiquement le résultat.
Voici deux exemples de résultats obtenus pour le Horla de Maupassant et Les misérables de V Hugo.
<img src="HorlalongV.png">
<img src="MiserableslongV1.png">

<b>Remarque :</b> Le code permettant le tracé des courbes est présenté en [annexe](#annexe).

### 2. Taille du dictionnaire vs taille du texte

La taille du texte ne semble pas avoir d’effet sur le taux de compression mais il est nécessaire de fournir le dictionnaire avec le texte compressé.
Dès lors la longueur du texte doit être prise en compte. Si le texte est court, la taille du dictionnaire est significative, mais pour un texte long, cette longueur est négligeable.

|Roman|Taille initiale(bits)|Taille compressée(bits)|Taille dictionnaire (bit)|Rapport taille dico/ taille code|
|:---:|:-------------------:|:---------------------:|:-----------------------------:|:-----------------------:|
|4 fables|36360|20416|1171|5,7 %|
|Candide|1611640|917975|1717|0,19%|
|Le Horla|2130432|1231817|1851|0,15%|
|Intégrale des fables|4272944|2378855|2039|0,086 %|
|Guerre et Paix|23187896|13177059|2088|0,015 %|
|Les misérables|26010384|14838532|2428|0,016 %|
|3 tomes de la comédie humaine|27394856|15409650|2118|0,014 %|


<b>Remarque :</b>

La taille du dictionnaire a été obtenu par le calcul suivant :
$ taille = 8n + n . nb bits du codage $  en bits où n est le nombre total de caractères présents dans le dictionnaire.



In [42]:
def tailleDico_bit(dico):
    """Prend en entrée un dictionnaire du type(key:caractère/value:binaire)
    retourne un entier : la taille du dictionnaire en bits
    """
    acc=0
    for key,value in dico.items():
        acc=acc+8+len(value)
    return acc

dicoSortie={'e': '00', 'l': '010', 'p': '0110', 'n': '0111', ' ': '100', 'j': '1010', 't': '1011', 'o': '1100', 'r': '1101', 'u': '11100', 'i': '11101', ',': '11110', 'c': '11111'}
print("la taille du dictinnaire est de",tailleDico_bit(dicoSortie),"bits")


la taille du dictinnaire est de 156 bits


### 3. Influence du nombre de caractères différents composant un texte.

Tous ces textes sont à base d’une centaine de caractères différents.
Il peut être intéressant de regarder un texte composé d’un petit nombre de caractères.
Pour cela nous avons généré un texte aléatoire avec la fonction suivante :

In [43]:
def generateurtexte(longueur,nbrcaracdiff):
    """
    longueur du texte,nombre de caractères différents --> texte
    """
    from random import randint
    texte=""
    alphareduit=[]
    alphabet=[]    
    for c in range(65,91):    
        alphabet.append(chr(c))
    for c in range(97,122):
        alphabet.append(chr(c))
    for c in [32,46,58,59]:
        alphabet.append(chr(c))
    alphareduit=alphabet[0:nbrcaracdiff]
    
    for i in range(0,longueur):
        c=randint(0,len(alphareduit)-1)
        texte=texte+alphareduit[c]
    return texte
#Pour exemple génération d'un texte
print(generateurtexte(200,16))

LANPIHIGJNMAKGBNBEGOJDPICBDNIHONPIOMNHIOJEHPBAGCPFGHHNCOFNFEHGFEDCBNPKEHJPHNLMMMJKBDMGGDKDJNGCHJALOGLIJDACDNKNEIFGKILNOGMHENMCIOIMBBPCGNJNMHGJIIAMAKKOHPBBCBPIMJMIINJPLEDEAOBAAHMJHONLELCKMJNLBJELJDOOAN


On peut grâce à python facilement tracer des graphiques montrant la duré du traitement pour un nombre de caractères définis
<img src="traitement8clongV.png">

On voit que la durée du traitement augmente de façon quasi-linéaire avec le nombre de caractères différents composant le texte. 

<img src="traitement128longV.png">


On retrouve bien la complexité quasi-linéaire rencontrée lors de l'étude de la compression 

Pour une longueur donnée on peut regarder comment évolue la durée en fonction du nombre de caractères composant le texte.
<img src="traitementlong1000cV.png">
<img src="traitement1milloncV.png">

On retrouve bien la complexité quasi-linéaire rencontrée lors de l'étude de la compression 

<b>Remarque :</b> Le code permettant le tracé des courbes est présenté en [annexe](#annexe).

# IV Conclusion

L’approche du code d’Huffman nous a permis de voir des méthodes de compression utilisant les tas ou les listes et de les  comparer.
Leurs complexités diffèrent, et il serait intéressant d’approfondir l’étude pour mieux faire apparaître cette différence.
L’étude de la compression des textes montre une complexité quasi linéaire qui correspond bien à l’étude théorique.
La complexité quadratique de la compression en utilisant les listes n’est pas aussi probante. Il faudrait approfondir ce travail afin de voir l’origine de l'écart entre l'étude théorique et les graphiques issues de la mise en pratique.
En effet quelque soit la méthode utilisée ( liste ou tas) les résultats sont similaires et se rapprochent d’une complexité linéaire. Cela est contradictoire avec notre étude théorique sur les listes.

<img src="ComparaisonMiserables.png">
<center><b>
 (+) : liste                   (°) : tas
    </b></center>
<img src="ComparaisonEnsemlbesRomans.png">

La décompression nous a montré l’utilité des arbres et rend simple la compréhension de cette étape.

Ce projet nous a aussi permis encore une fois de voir les possibilités offertes par le langage python, langage relativement nouveau pour nous. Les bibliothèques et les ressources nombreuses permettent de se sortir aisément des difficultés rencontrées.

Enfin il pourrait être intéressant pour une utilisation pédagogique de rendre plus graphique ce projet. Cela pourrait passer par un codage autre que les caractères de l’alphabet. Le code d’Huffman est en effet utilisé dans la compression d’images.

# V Annexes<div id ="annexe">

Afin de mieux suivre les étapes de la compression et de la décompression, de faire des tracés de courbes nous avons utilisé des fonctions nous aidant à utiliser le programme créé.
En voici une présentation de certaines.


In [44]:
def messageEntre():
    """
    Fonction qui permet de récupérer en sortie une chaîne de caractères issue 
    soit d'un texte tapé par l'utilisateur
    soit d'un fichier.txt dont le nom est donné par l'utilisateur    
    """
    print("codage de Huffman d'un texte")
    print("voulez-vous :")
    print("1 -taper un texte")
    print("2 -entrer le nom du fichier")
    reponse=input()
    message=""
    if reponse=="1":
        message=input("taper votre texte et appuyer sur entrée : ")
    if reponse=="2":
        nom=input("entrer le nom du fichier : ")# fichier de départ contenant le texte
        file=open(nom,'r',encoding="UTF-8") #ouverture du fichier
        message = file.read() #stockage dans la variable texte du fichier
        file.close() # fermeture du fichier
    return message
messageEntre()

codage de Huffman d'un texte
voulez-vous :
1 -taper un texte
2 -entrer le nom du fichier
1
taper votre texte et appuyer sur entrée : 3lg


'3lg'

Pour la mesure des durées de traitement des textes nous avons utilisé la bibliothèque time

In [49]:
import time
texte="en juillet, encore le projet"
start_time1=time.time()

tas={}
dicoT=creationDictionnaireOccurence(texte)
tas=conversionDictionnaireOccurenceEnTas(dicoT)

arbreT = faireArbre(tas)

dicoSortieT = code_huffman(arbreT)
binaireT = encodage(dicoSortieT,texte)
stop_time1=time.time()

"""
resultat faisable en une ligne
binaire = encodage(code_huffman(faireArbreAvecListe(conversionDictionnaireOccurenceEnListe(creationDictionnaireOccurence(texte)))),texte)
"""
start_time2=time.time()
dicoL=creationDictionnaireOccurence(texte)
liste=conversionDictionnaireOccurenceEnListe(dicoL)
arbreL = faireArbreAvecListe(liste)
dicoSortieL = code_huffman(arbreL)
binaireL = encodage(dicoSortieL,texte)
stop_time2=time.time()

print("----------------statistiques avec les tas----------------")
print("---------------------------------------------------------")
print("Temps d execution : %s secondes" % (stop_time1 - start_time1))
print("----------------statistiques avec les listes----------------")
print("------------------------------------------------------------")
print("Temps d execution : %s secondes" % (stop_time2-start_time2))
print("---------------------------------------------------------")
print("---------------------------------------------------------")
print("longueur du texte :", len(texte)*8,'bits'," longueur du message codé : ", len(binaireL),'bits')
print("taux de compression : ", int(len(binaireL)/(8*len(texte))*100))
print("taille du dico :",tailleDico_bit(dicoSortieL)," bits")
print("nombre de caractères:",len(dicoT))
print("nombre de bits pour le codage:", nombre2bitsutilisés(dicoSortieL))


NameError: name 'creationDictionnaireOccurence' is not defined

Enfin pour le tracé des courbes montrant la durée de traitement en fonction de la longueur d'un texte ou pour afficher le taux de compression nous avons utilisé les bibliothèques matplotlib.

Les fichiers texte ont été pris sur http://www.gutenberg.org/, nous avons essayé de trouver des textes de longueurs différentes.
* 4 fables de la Fontaine
* Le Horla de Guy de Maupassant 
* Les Miserables de Vicor Hugo 
* Guerre et Paix de Léon Tolstoï 
* Candide de Voltaire
* L'integrales des fables de la Fontaine
* 3 tomes de la Comédie Humaine de Balzac

Pour le fonctionnenemnt des programmes, ces fichiers textes doivent être placés dans le même répertoire que le notebook.

In [50]:
def generateurTexte(longueur,Nbca):
    """
    Fonction qui prend un entier longueur et un entier Nbca
    Retourne un texte de longueur longueur composé Nbca caractères différents
    """
    texte=""
    choixLettre=alphabet[0:Nbca]
    for i in range(longueur):
        texte=texte+choixLettre[randint(0,len(choixLettre)-1)]
    return texte

def troncageTexte(texte,longueur):
    """prend une chaîne de caractères texte et un entier longueur
    retourne les n premiers caractères de la chaîne"""
    texteCoupe=texte[0:longueur]
    return texteCoupe

def encodageGlobalTas(texte):
    return encodage(code_huffman(faireArbre(conversionDictionnaireOccurenceEnTas(creationDictionnaireOccurence(texte)))),texte)
    
def encodageGlobalListe(texte):
    return encodage(code_huffman(faireArbreAvecListe(conversionDictionnaireOccurenceEnListe(creationDictionnaireOccurence(texte)))),texte)

def tracerCourbe(x,y,couleur):
    pyplot.scatter(x, y, c = couleur)
    return None

###########################################################################
################       Programme Principal                #################
###########################################################################

###
#EVOLUTION
###
from matplotlib import pyplot
from random import randint
from random import shuffle
import time

alphabet=[]
for i in range(65,91):
    alphabet.append(chr(i))
for i in range(97,123):
    alphabet.append(chr(i))
shuffle(alphabet)
alphabet.insert(0," ")

durée=[]
compression=[]
longueurTexte=[]
xlong=[]

for nom in ["leHorla.txt","LesMiserables.txt","GuerrePaix.txt","candide.txt","IntegralesFables.txt","4fables.txt","Balzac.txt"]:
    file=open(nom,'r',encoding="UTF-8") #ouverture du fichier
    message = file.read() #stockage dans la variable texte du fichier
    file.close() # fermeture du fichier
    taille=len(message)
    for i in range(1,21):
        longueurTexte.append(int(taille//i))    
    for long in longueurTexte:
        texte=troncageTexte(message,long)
        start=time.time()
        code=encodageGlobalListe(texte)
        stop=time.time()
        xlong.append(long)
        durée.append(stop-start)
        compression.append((len(code)/(8*len(texte))*100))
    
#tracerCourbe(xlong,durée,"green")
tracerCourbe(xlong,durée,"red")
#pyplot.xscale('log')
#pyplot.axis([0, taille, 0, 100])
pyplot.xlabel('Longueur du texte')
pyplot.ylabel('durée')
pyplot.title("Durée pour l'ensemble des livres de 10 à 100%)")
pyplot.show()

FileNotFoundError: [Errno 2] No such file or directory: 'leHorla.txt'